<a href="https://colab.research.google.com/github/dzejkopjarosinski/ZespolR/blob/main/MLF_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importy**

In [1]:
!pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173804 sha256=c93dc629870009583904acd76347aab00f241df1b67c617a5541c28a56e4171f
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [2]:
#Modelowanie
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from scipy.optimize import minimize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
import yfinance as yf

# Dane gtowe

In [3]:
# SP500 prices from 2003-08 to 2014-12-31
sp500_index = pd.read_csv('sp500_index.csv')

#SP500 individual stock prices from 2003-08 to 2009-02
stock_prices = pd.read_csv('stock_prices.csv')

#Keystats from 2003-08 to 2013-09
keystats = pd.read_csv('keystats.csv')

#forward_sample
forward_sample = pd.read_csv('forward_sample.csv')

In [4]:
stock_prices.head()

,Date,A,AA,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XEL,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZION,ZNGA
0,2003-07-31,13.790836,53.019848,1.019303,NaN,12.739487,9.022574,15.108545,16.333433,26.962475,...,7.946304,53.450195,19.403030,24.283438,20.140688,23.011992,NaN,8.275629,46.391441,NaN
1,2003-08-01,13.676599,51.759747,1.002379,NaN,12.579965,8.859362,15.816516,16.268559,28.070812,...,7.858501,50.491951,19.898077,24.112812,19.770533,22.692390,NaN,8.272863,46.557457,NaN
2,2003-08-04,13.682946,52.256149,1.025589,NaN,12.398233,8.850165,15.660917,16.523066,28.227112,...,7.803621,51.500431,19.838972,24.303909,19.728891,21.605703,NaN,8.239684,46.673660,NaN
3,2003-08-05,13.219654,51.206055,0.985455,NaN,12.111500,8.700747,15.147444,16.183720,27.132978,...,7.699356,51.856777,19.225708,24.126451,19.405010,21.605703,NaN,8.167795,46.325050,NaN
4,2003-08-06,13.060995,50.783653,0.949190,NaN,11.752071,8.873152,14.937387,15.749559,27.175615,...,7.748745,52.643398,19.151808,24.399456,19.423517,21.818781,NaN,8.037839,46.648762,NaN


In [5]:
sp500_index.Date.min()

'2003-07-31'

In [6]:
keystats.head()

,Date,Unix,Ticker,Price,stock_p_change,SP500,SP500_p_change,Market Cap,Enterprise Value,Trailing P/E,...,200-Day Moving Average,Avg Vol (3 month),Shares Outstanding,Float,% Held by Insiders,% Held by Institutions,Shares Short (as of,Short Ratio,Short % of Float,Shares Short (prior month)
0,2007-07-11 05:59:28,1.184105e+09,ctas,33.538876,-33.02,122.084023,-15.90,6.350000e+09,7.080000e+09,19.13,...,39.30,1064470.0,158650000.0,135580000.0,17.81,64.70,4970000.0,4.200,3.70,4490000.0
1,2006-02-28 20:38:59,1.141130e+09,ctas,34.026257,-0.79,100.297188,11.94,6.910000e+09,7.160000e+09,22.55,...,41.44,940276.0,168020000.0,140190000.0,16.59,66.20,3460000.0,3.400,2.50,4600000.0
2,2005-04-20 15:38:44,1.113983e+09,ctas,31.885536,9.65,87.811470,17.27,6.750000e+09,7.020000e+09,23.35,...,43.13,1027318.0,172090000.0,122500000.0,28.82,65.87,6170000.0,6.479,5.04,6360000.0
3,2005-02-22 13:02:43,1.109049e+09,ctas,35.317966,-4.10,91.157219,10.92,7.550000e+09,7.670000e+09,26.61,...,43.75,994136.0,171720000.0,122200000.0,28.84,63.20,5420000.0,4.740,4.44,4640000.0
4,2013-04-26 17:42:40,1.366969e+09,ctas,41.474506,30.78,143.779022,20.10,5.480000e+09,6.550000e+09,18.46,...,42.70,580981.0,122520000.0,102370000.0,16.36,74.60,5060000.0,8.000,5.60,5860000.0


#Model

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:

training_data = keystats.set_index('Date')
training_data.dropna(axis=0, how='any', inplace=True)
features = training_data.columns[6:]

X_train = training_data[features].values

OUTPERFORMANCE = 10

y_train = [
  1 if stock_p_change - SP500_p_change >= OUTPERFORMANCE else 0
  for stock_p_change, SP500_p_change in zip(
      training_data["stock_p_change"], training_data["SP500_p_change"]
  )]



In [12]:
rfc = RandomForestClassifier(n_estimators = 100, random_state=0)
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [13]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [14]:
# Feature List per Ticker
forward_sample = forward_sample.set_index('Date')
forward_sample.dropna(axis=0, how='any', inplace=True)
features = forward_sample.columns[6:]
X_test = forward_sample[features].values

#Lista tickerów
z = forward_sample['Ticker'].values

In [15]:
y_pred = rfc.predict(X_test)

#y_pred.map(lambda x : True if 1 False if 0)
y_pred_b = y_pred.astype(bool)

if sum(y_pred) == 0:
      print("No stocks predicted!")
else:
  invest_list = z[y_pred_b].tolist()
  print(
        f"{len(invest_list)} stocks predicted to outperform the S&P500 by more than {OUTPERFORMANCE}%:"
        )
  print(" ".join(invest_list))


34 stocks predicted to outperform the S&P500 by more than 10%:
ABC LSI TRV IP PH OI BAX FDX INTU SWK WGO DGX LH SNA GT BIIB DHI BWA GES GNW DNR LLL LYB R PBI BLK DLX DIS GTN AMP LMT LM PHM APD


In [16]:
y_pred2 = knn.predict(X_test)

#y_pred.map(lambda x : True if 1 False if 0)
y_pred2_b = y_pred.astype(bool)

if sum(y_pred2) == 0:
      print("No stocks predicted!")
else:
  invest_list = z[y_pred2_b].tolist()
  print(
        f"{len(invest_list)} stocks predicted to outperform the S&P500 by more than {OUTPERFORMANCE}%:"
        )
  print(" ".join(invest_list))

34 stocks predicted to outperform the S&P500 by more than 10%:
ABC LSI TRV IP PH OI BAX FDX INTU SWK WGO DGX LH SNA GT BIIB DHI BWA GES GNW DNR LLL LYB R PBI BLK DLX DIS GTN AMP LMT LM PHM APD


# BackTesting - Lib

In [17]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [18]:
stock_prices.set_index('Date', inplace=True)
stock_prices.index = pd.to_datetime(stock_prices.index)
stock_prices

,A,AA,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,...,XEL,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZION,ZNGA
Date,,,,,,,,,,,,,,,,,,,,,
2003-07-31,13.790836,53.019848,1.019303,NaN,12.739487,9.022574,15.108545,16.333433,26.962475,9.694519,...,7.946304,53.450195,19.403030,24.283438,20.140688,23.011992,NaN,8.275629,46.391441,NaN
2003-08-01,13.676599,51.759747,1.002379,NaN,12.579965,8.859362,15.816516,16.268559,28.070812,9.561718,...,7.858501,50.491951,19.898077,24.112812,19.770533,22.692390,NaN,8.272863,46.557457,NaN
2003-08-04,13.682946,52.256149,1.025589,NaN,12.398233,8.850165,15.660917,16.523066,28.227112,9.591229,...,7.803621,51.500431,19.838972,24.303909,19.728891,21.605703,NaN,8.239684,46.673660,NaN
2003-08-05,13.219654,51.206055,0.985455,NaN,12.111500,8.700747,15.147444,16.183720,27.132978,9.532207,...,7.699356,51.856777,19.225708,24.126451,19.405010,21.605703,NaN,8.167795,46.325050,NaN
2003-08-06,13.060995,50.783653,0.949190,NaN,11.752071,8.873152,14.937387,15.749559,27.175615,9.620744,...,7.748745,52.643398,19.151808,24.399456,19.423517,21.818781,NaN,8.037839,46.648762,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-04-29,20.584120,73.059105,16.928751,NaN,17.924480,16.796642,31.191916,37.759998,24.528233,35.998608,...,14.060099,26.535021,19.482054,69.084290,36.522018,30.080030,NaN,23.731174,43.007530,NaN
2008-04-30,20.348377,73.694763,16.822374,NaN,17.467855,16.567369,30.210373,37.290001,24.361841,34.798126,...,14.080407,26.542627,19.256601,70.047653,36.758419,29.930082,NaN,23.615070,42.566696,NaN
2008-05-01,20.947847,73.567673,17.407454,NaN,17.855556,16.824909,30.878145,39.009998,25.413157,34.111008,...,14.378263,27.881554,19.932955,67.511299,38.734886,31.022709,NaN,23.678923,45.312641,NaN


In [19]:
sp500_data = sp500_index
data_stocks = stock_prices[invest_list]
prediction = invest_list

In [20]:
data_stocks = data_stocks.sum(axis=1)
data_stocks = pd.DataFrame({'Close': data_stocks})

In [21]:
data_stocks.head()

,Close
Date,
2003-07-31,655.043378
2003-08-01,653.055012
2003-08-04,651.426360
2003-08-05,641.488822
2003-08-06,641.944422


In [ ]:
#data_stocks = data_stocks.stack().reset_index()
#data_stocks.columns = ['Date', 'Ticker', 'Close']

In [22]:
data_stocks['Open'] = data_stocks['Close']
data_stocks['High'] = data_stocks['Close']
data_stocks['Low'] = data_stocks['Close']
data_stocks['Volume'] = 0

In [23]:
data_stocks.head()

,Close,Open,High,Low,Volume
Date,,,,,
2003-07-31,655.043378,655.043378,655.043378,655.043378,0
2003-08-01,653.055012,653.055012,653.055012,653.055012,0
2003-08-04,651.426360,651.426360,651.426360,651.426360,0
2003-08-05,641.488822,641.488822,641.488822,641.488822,0
2003-08-06,641.944422,641.944422,641.944422,641.944422,0


In [26]:
data_stocks

,Close,Open,High,Low,Volume
Date,,,,,
2003-07-31,655.043378,655.043378,655.043378,655.043378,0
2003-08-01,653.055012,653.055012,653.055012,653.055012,0
2003-08-04,651.426360,651.426360,651.426360,651.426360,0
2003-08-05,641.488822,641.488822,641.488822,641.488822,0
2003-08-06,641.944422,641.944422,641.944422,641.944422,0
...,...,...,...,...,...
2008-04-29,1401.042295,1401.042295,1401.042295,1401.042295,0
2008-04-30,1394.658951,1394.658951,1394.658951,1394.658951,0
2008-05-01,1423.866555,1423.866555,1423.866555,1423.866555,0


In [ ]:
# Define your custom strategy
class MyStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        if crossover(self.data.Close, self.data.Open):
            self.buy()


# Run the backtest
bt = Backtest(data_stocks, MyStrategy)

# View the performance metrics and equity curve
results = bt.run()
print(results)
bt.plot()

Start                     2003-07-31 00:00:00
End                       2014-12-31 00:00:00
Duration                   4171 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]              345.823646
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

Row(id='1172', ...)

In [ ]:
class BasketStrategy(Strategy):
    def init(self):
        # Store the selected tickers separately
        self.tickers = set(self.data['Ticker'])
    
    def next(self):
        for ticker in self.tickers:
            # Filter the data for the specific ticker
            ticker_data = self.data[self.data['Ticker'] == ticker]
            
            if len(ticker_data) >= 2:
                close_prices = ticker_data['Close']
                
                if close_prices[-2] < close_prices[-1]:
                    self.buy(ticker=ticker)
                elif close_prices[-2] > close_prices[-1]:
                    self.sell(ticker=ticker)




In [ ]:
bt = Backtest(data_stocks, BasketStrategy)
results = bt.run()

print(results)
bt.plot()


KeyError: ignored

In [ ]:
class MyStrategy(Strategy):
    def __init__(self):
        self.stocks_to_buy = prediction
    
    def next(self):
        if self.data_stocks.index.year == 2005:
            for stock in self.stocks_to_buy:
                if stock in self.data_stocks.columns:  # Check if the stock is available in the data
                        self.buy(stock, 1)  # Buy one share of the stock
'''
    def next_open(self):
        # Kup pod koniec 2005 (następny rok to 2006 a następny miesiąc to styczeń)
        if self.data_stocks.index[-1].year == 2006 and self.data_stocks.index[-1].month == 1:
            for stock in self.stocks_to_buy:
                self.buy(stock, 1)  # Buy 1 share of each stock
'''

'\n    def next_open(self):\n        # Kup pod koniec 2005 (następny rok to 2006 a następny miesiąc to styczeń)\n        if self.data_stocks.index[-1].year == 2006 and self.data_stocks.index[-1].month == 1:\n            for stock in self.stocks_to_buy:\n                self.buy(stock, 1)  # Buy 1 share of each stock\n'

In [ ]:
bt = Backtest(data_stocks, MyStrategy)
results = bt.run()
print(results)

TypeError: ignored

# BackTesting

In [27]:
from sklearn.metrics import precision_score

In [29]:
def status_calc(stock, sp500, outperformance=10):
    if outperformance < 0:
        raise ValueError("outperformance must be positive")
    return stock - sp500 >= outperformance

In [30]:
def backtest():

    data_df = pd.read_csv("keystats.csv", index_col="Date")
    data_df.dropna(axis=0, how="any", inplace=True)

    features = data_df.columns[6:]
    X = data_df[features].values

    y = list(
        status_calc(
            data_df["stock_p_change"], data_df["SP500_p_change"], outperformance=10
        )
    )

    # z is required for us to track returns
    z = np.array(data_df[["stock_p_change", "SP500_p_change"]])

    # Generate the train set and test set by randomly splitting the dataset
    X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(
        X, y, z, test_size=0.2
    )

    # Instantiate a RandomForestClassifier with 100 trees, then fit it to the training data
    clf = RandomForestClassifier(n_estimators=100, random_state=0)
    clf.fit(X_train, y_train)

    # Generate the predictions, then print test set accuracy and precision
    y_pred = clf.predict(X_test)
    print("Classifier performance\n", "=" * 20)
    print(f"Accuracy score: {clf.score(X_test, y_test): .2f}")
    print(f"Precision score: {precision_score(y_test, y_pred): .2f}")

   
    num_positive_predictions = sum(y_pred)
    if num_positive_predictions < 0:
        print("No stocks predicted!")
    stock_returns = 1 + z_test[y_pred, 0] / 100
    market_returns = 1 + z_test[y_pred, 1] / 100

    avg_predicted_stock_growth = sum(stock_returns) / num_positive_predictions
    index_growth = sum(market_returns) / num_positive_predictions
    percentage_stock_returns = 100 * (avg_predicted_stock_growth - 1)
    percentage_market_returns = 100 * (index_growth - 1)
    total_outperformance = percentage_stock_returns - percentage_market_returns

    print("\n Stock prediction performance report \n", "=" * 40)
    print(f"Total Trades:", num_positive_predictions)
    print(f"Average return for stock predictions: {percentage_stock_returns: .1f} %")
    print(
        f"Average market return in the same period: {percentage_market_returns: .1f}% "
    )
    print(
        f"Compared to the index, our strategy earns {total_outperformance: .1f} percentage points more"
    )


if __name__ == "__main__":
    backtest()

Classifier performance
Accuracy score:  0.79
Precision score:  0.75

 Stock prediction performance report 
Total Trades: 182
Average return for stock predictions:  38.3 %
Average market return in the same period:  10.1% 
Compared to the index, our strategy earns  28.1 percentage points more


In [31]:
def backtest():

    data_df = pd.read_csv("keystats.csv", index_col="Date")
    data_df.dropna(axis=0, how="any", inplace=True)

    features = data_df.columns[6:]
    X = data_df[features].values

    y = list(
        status_calc(
            data_df["stock_p_change"], data_df["SP500_p_change"], outperformance=10
        )
    )

    # z is required for us to track returns
    z = np.array(data_df[["stock_p_change", "SP500_p_change"]])

    # Generate the train set and test set by randomly splitting the dataset
    X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(
        X, y, z, test_size=0.2
    )

    # Instantiate a RandomForestClassifier with 100 trees, then fit it to the training data
    clf = KNeighborsClassifier(n_neighbors=3)
    clf.fit(X_train, y_train)

    # Generate the predictions, then print test set accuracy and precision
    y_pred = clf.predict(X_test)
    print("Classifier performance\n", "=" * 20)
    print(f"Accuracy score: {clf.score(X_test, y_test): .2f}")
    print(f"Precision score: {precision_score(y_test, y_pred): .2f}")

   
    num_positive_predictions = sum(y_pred)
    if num_positive_predictions < 0:
        print("No stocks predicted!")
    stock_returns = 1 + z_test[y_pred, 0] / 100
    market_returns = 1 + z_test[y_pred, 1] / 100

    avg_predicted_stock_growth = sum(stock_returns) / num_positive_predictions
    index_growth = sum(market_returns) / num_positive_predictions
    percentage_stock_returns = 100 * (avg_predicted_stock_growth - 1)
    percentage_market_returns = 100 * (index_growth - 1)
    total_outperformance = percentage_stock_returns - percentage_market_returns

    print("\n Stock prediction performance report \n", "=" * 40)
    print(f"Total Trades:", num_positive_predictions)
    print(f"Average return for stock predictions: {percentage_stock_returns: .1f} %")
    print(
        f"Average market return in the same period: {percentage_market_returns: .1f}% "
    )
    print(
        f"Compared to the index, our strategy earns {total_outperformance: .1f} percentage points more"
    )


if __name__ == "__main__":
    backtest()

Classifier performance
Accuracy score:  0.72
Precision score:  0.59

 Stock prediction performance report 
Total Trades: 198
Average return for stock predictions:  25.4 %
Average market return in the same period:  7.9% 
Compared to the index, our strategy earns  17.6 percentage points more
